# Data cleaning process for the Provider Suspended and Ineligible List
## Creates 2 final datasets that are saved to CSV to be fed directly to my Streamlit app

In [86]:
import pandas as pd
import time
import re
#!pip install altair vega_datasets
#import altair as alt
#!pip install folium
#import folium
#!pip install streamlit-folium
#from streamlit_folium import folium_static


  Obtaining dependency information for streamlit-folium from https://files.pythonhosted.org/packages/57/69/13e81b032db6ccdb38cd4433b84808b3a580942e0c23bb89e15e9399c697/streamlit_folium-0.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for tzdata from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/326.0 kB ? eta -:--:--
   -------------------------------------- - 317.4/326.0 kB 6.5 MB/s eta 0:00:01
   -------------------------------------- - 317.4/326.0 kB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 326.0/326.0 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/345.4 kB ? eta -:--:--
   --------------------------------------  337.9/345.4 kB 20.5 MB/s eta 0:00:01
   ---------------------------------------- 345.4/345.4 kB 5.3 MB/s eta 0:00:00


In [46]:
df = pd.read_csv("provider-suspended-and-ineligible-list-s-i-list.csv", encoding="windows-1252")


In [47]:
def extract_address_with_zip_or_state(address):
    zip_code_match = re.search(r'.*?\b(\d{5})', address)
    if zip_code_match:
        #Return the address up to and including the 5-digit zip code
        return zip_code_match.group(0)
    else:
        # If no zip code is found, look for two capital letters representing the state code
        state_code_match = re.search(r'.*?\b([A-Z]{2})', address)
        if state_code_match:
            # Return the address up to and including the state code
            return state_code_match.group(0)
        else:
            # If no state code is found, return the original address
            return address

# Apply the function to create the "1st Address" column
df['1st Address'] = df['Address(es)'].apply(extract_address_with_zip_or_state)

# Extracting city and zip code
df['City'] = df['1st Address'].str.extract('([^,]+), [A-Z]{2},? ?\d*$', expand=False)
df['Zip Code'] = df['1st Address'].str.extract('(\d{5})$', expand=False)

# Handling special cases where the address might not follow the pattern
df['City'] = df['City'].fillna('Unknown')  # Filling NaN values for City with 'Unknown'
df['Zip Code'] = df['Zip Code'].fillna('Unknown')  # Filling NaN values for Zip Code with 'Unknown'


,Last Name,First Name,Middle Name,A/K/A-Also Known As\nD/B/A-Doing Business as,Address(es),Provider Type,License Number,Provider Number,Date of Suspension,Active Period,1st Address,City,Zip Code
0,111 Pharmacy,NaN,NaN,NaN,"111 West Beverly Blvd., Ste. B, Montebello, CA...",Pharmacy/Pharmacist,41023,"PHA410230, PHA393440, 1821137118",7/8/2016,Indefinitely effective,"111 West Beverly Blvd., Ste. B, Montebello, CA...",Montebello,90640
1,"2nd Chance for Recovery, Inc.",NaN,NaN,NaN,"600 East 7th St., Los Angeles, CA, 90021",Entity/Corporation,NaN,"1003077710, 1033522974, 1346589777, 1760751606",05/23/2018,Indefinitely effective,"600 East 7th St., Los Angeles, CA, 90021",Los Angeles,90021
2,3 Angeles Medical Clinic,NaN,NaN,NaN,"333 Wilkerson Ave., Stes. B & C, Perris, CA; 4...",Clinic,NaN,NaN,3/11/2009,Indefinitely effective,"333 Wilkerson Ave., Stes. B & C, Perris, CA",Perris,Unknown
3,52 Street Pharmacy,NaN,NaN,NaN,"5231 El Cajon Blvd., San Diego, CA,",Pharmacy,NaN,"PHA118810, PHA200040, PHA217110, CGP14678",12/26/1990,Indefinitely effective,"5231 El Cajon Blvd., San Diego, CA",San Diego,Unknown
4,A & R Medical Transport,NaN,NaN,"Aghajanyan, Mike; Aghajanyan, Roland; Aghajany...","5300 Santa Monica Blvd., Ste. #216, Los Angele...",Medical Transportation/Owner,NaN,MTN01203F,7/4/2012,Indefinitely effective,"5300 Santa Monica Blvd., Ste. #216, Los Angele...",Los Angeles,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21631,Wehrly,John,Dale,NaN,"154 Ely Street, Oceanside, CA, 92054",Registered Nurse,484699,NaN,06/15/2023,Indefinitely effective,"154 Ely Street, Oceanside, CA, 92054",Oceanside,92054
21632,Weinberger,Alan,Walter,NaN,"P.O. Box 676, South Pasadena, CA, 91031",Physician,G 33359,"00G333590, 1124015011",05/18/2023,Indefinitely effective,"P.O. Box 676, South Pasadena, CA, 91031",South Pasadena,91031
21633,Winters,Julie,Mae,NaN,"6024 Fyler Avenue, Saint Louis, MO, 63139",Registered Nurse,774207,NaN,06/16/2023,Indefinitely effective,"6024 Fyler Avenue, Saint Louis, MO, 63139",Saint Louis,63139
21634,Dondiego,Madelaine,NaN,NaN,"Delano, CA",In-Home Supportive Services Provider,1336006,NaN,07/11/2023,Indefinitely effective,"Delano, CA",Delano,Unknown


In [48]:
def clean_suite_apt_info(address):
    # Count the number of commas in the address
    comma_count = address.count(',')
    
    # If there are exactly 3 commas, remove text between the first and second comma
    if comma_count == 3:
        parts = address.split(',')
        # Reconstruct the address without the part between the first and second comma
        new_address = parts[0] + ',' + parts[2] + ',' + parts[3]
        return new_address
    else:
        # If the condition doesn't match, return the address as is
        return address

# Apply the function to clean apartment or suite info from "1st Address"
df['Cleaned Address'] = df['1st Address'].apply(clean_suite_apt_info)



In [43]:
import requests
import pandas as pd

# Example address without a zip code
addresses = ["333 Wilkerson Ave., Perris, CA"]

# Function to get zip code using Nominatim API
def get_zip_code(address):
    params = {
        'q': address,
        'format': 'json'
    }
    response = requests.get('https://nominatim.openstreetmap.org/search', params=params)
    data = response.json()
    if data:
        # Assuming the first result is the most relevant one
        return data[0].get('display_name').split(',')[-2].strip()
    return None

# Loop through addresses and get zip codes
for address in addresses:
    zip_code = get_zip_code(address)
    print(f'Address: {address}, Zip Code: {zip_code}')
    

Address: 333 Wilkerson Ave., Perris, CA, Zip Code: 92570


In [50]:
import requests
import pandas as pd

def get_zip_code(address):
    """Get the zip code for an address using the Nominatim API."""
    params = {
        'q': address,
        'format': 'json'
    }
    response = requests.get('https://nominatim.openstreetmap.org/search', params=params)
    data = response.json()
    if data:
        # Assuming the first result is the most relevant one
        # Attempt to extract the zip code more reliably
        address_components = data[0].get('display_name').split(',')
        zip_code = [comp.strip() for comp in address_components if comp.strip().isdigit() and len(comp.strip()) == 5]
        if zip_code:
            time.sleep(1)  # Respect Nominatim's usage policy
            return zip_code[0]
    return None

# Loop through addresses and get zip codes
for index, row in df[df['Zip Code'] == 'Unknown'].iterrows():
    address = row['Cleaned Address']
    zip_code = get_zip_code(address)
    if zip_code:
        df.at[index, 'Zip Code'] = zip_code  # Correctly update the zip code for the specific row
        print(f'Address: {address}, Zip Code: {zip_code}')

Address: 333 Wilkerson Ave., Perris, CA, Zip Code: 92570
Address: 5231 El Cajon Blvd., San Diego, CA, Zip Code: 92115
Address: 5300 Santa Monica Blvd., Los Angeles, CA, Zip Code: 90029
Address: 3901 Ursula Ave., Los Angeles, CA, Zip Code: 90008
Address: 4708 Crenshaw Blvd., Los Angeles, CA, Zip Code: 90043
Address: 1421 Oakdale Rd., Modesto, CA, Zip Code: 95355
Address: P.O. Box 4, Biola, CA, Zip Code: 93606
Address: 1375 Park Ln., Bullhead City, AZ, Zip Code: 86442
Address: 2406 White Ridge Ln., Washington, IN, Zip Code: 47501
Address: 824 South Alvarado St., Los Angeles, CA, Zip Code: 90057
Address: 7409 Irvine Ave., North Hollywood, CA, Zip Code: 91606
Address: 344 St. Joseph St., New Orleans, LA, Zip Code: 70130
Address: 5472 El Cajon Blvd., San Diego, CA, Zip Code: 92115
Address: 711 Diablo Ave., Novato, CA, Zip Code: 94947
Address: 1315 North 18th, Tacoma, WA, Zip Code: 98416
Address: 1496 Solano Ave., Albany, CA, Zip Code: 94707
Address: 1180 North Indian Canyon Dr., Palm Spring

KeyboardInterrupt: 

In [57]:
def get_zip_code(address):
    """Get the zip code for an address using the Nominatim API."""
    params = {
        'q': address,
        'format': 'json'
    }
    response = requests.get('https://nominatim.openstreetmap.org/search', params=params)
    data = response.json()
    if data:
        # Assuming the first result is the most relevant one
        # Attempt to extract the zip code more reliably
        address_components = data[0].get('display_name').split(',')
        zip_code = [comp.strip() for comp in address_components if comp.strip().isdigit() and len(comp.strip()) == 5]
        if zip_code:
            time.sleep(.2)  # Respect Nominatim's usage policy
            return zip_code[0]
    return None

# Loop through addresses and get zip codes
for index, row in df[df['Zip Code'] == 'Unknown'].iterrows():
    print(index)
    address = row['Cleaned Address']
    zip_code = get_zip_code(address)
    if zip_code:
        df.at[index, 'Zip Code'] = zip_code  # Correctly update the zip code for the specific row
        print(f'Address: {address}, Zip Code: {zip_code}')

10
32
53
59
105
108
118
119
126
131
132
142
154
156
172
190
192
193
199
209
210
236
261
264
278
290
311
341
355
357
371
373
374
406
407
414
420
429
433
434
436
437
440
466
471
487
491
512
514
518
536
552
576
611
613
614
625
627
635
641
653
659
667
696
703
706
734
741
743
747
757
762
765
792
796
811
819
839
888
890
894
895
903
945
953
968
976
987
992
1008
1043
1046
1063
1087
1097
1106
1118
1120
1123
1134
1143
1144
1150
1157
1172
1174
1178
1189
1190
1197
1201
1204
1207
1216
1239
1241
1246
1258
1267
1268
1279
1285
1297
1309
1331
1349
1366
1373
1374
1378
1387
1406
1408
1437
1445
1465
1490
1501
1509
1512
1513
1521
1522
1540
1549
1550
1577
1583
1589
1596
1607
1610
1615
1624
1628
1642
1647
1652
1690
1707
1735
1744
1766
1787
1800
1808
1826
1830
1833
1843
1844
1853
1854
1860
1877
1882
1883
1922
1930
1960
1968
1969
1980
1992
2008
2020
2025
2030
2036
2043
2044
2051
2053
2069
2080
2083
2090
2104
2112
2115
2117
2121
2129
2132
2134
2141
2147
2154
2172
2181
2189
2193
2195
2196
2232
2237
2248
2257
225

In [77]:
df['Provider Type'] = df['Provider Type'].apply(lambda x: str(x).split(',')[0].split(';')[0])
df

,Last Name,First Name,Middle Name,A/K/A-Also Known As\nD/B/A-Doing Business as,Address(es),Provider Type,License Number,Provider Number,Date of Suspension,Active Period,1st Address,City,Zip Code,Cleaned Address
0,111 Pharmacy,NaN,NaN,NaN,"111 West Beverly Blvd., Ste. B, Montebello, CA...",Pharmacy/Pharmacist,41023,"PHA410230, PHA393440, 1821137118",7/8/2016,Indefinitely effective,"111 West Beverly Blvd., Ste. B, Montebello, CA...",Montebello,90640,"111 West Beverly Blvd., Ste. B, Montebello, CA..."
1,"2nd Chance for Recovery, Inc.",NaN,NaN,NaN,"600 East 7th St., Los Angeles, CA, 90021",Entity/Corporation,NaN,"1003077710, 1033522974, 1346589777, 1760751606",05/23/2018,Indefinitely effective,"600 East 7th St., Los Angeles, CA, 90021",Los Angeles,90021,"600 East 7th St., CA, 90021"
2,3 Angeles Medical Clinic,NaN,NaN,NaN,"333 Wilkerson Ave., Stes. B & C, Perris, CA; 4...",Clinic,NaN,NaN,3/11/2009,Indefinitely effective,"333 Wilkerson Ave., Stes. B & C, Perris, CA",Perris,92570,"333 Wilkerson Ave., Perris, CA"
3,52 Street Pharmacy,NaN,NaN,NaN,"5231 El Cajon Blvd., San Diego, CA,",Pharmacy,NaN,"PHA118810, PHA200040, PHA217110, CGP14678",12/26/1990,Indefinitely effective,"5231 El Cajon Blvd., San Diego, CA",San Diego,92115,"5231 El Cajon Blvd., San Diego, CA"
4,A & R Medical Transport,NaN,NaN,"Aghajanyan, Mike; Aghajanyan, Roland; Aghajany...","5300 Santa Monica Blvd., Ste. #216, Los Angele...",Medical Transportation/Owner,NaN,MTN01203F,7/4/2012,Indefinitely effective,"5300 Santa Monica Blvd., Ste. #216, Los Angele...",Los Angeles,90029,"5300 Santa Monica Blvd., Los Angeles, CA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21631,Wehrly,John,Dale,NaN,"154 Ely Street, Oceanside, CA, 92054",Registered Nurse,484699,NaN,06/15/2023,Indefinitely effective,"154 Ely Street, Oceanside, CA, 92054",Oceanside,92054,"154 Ely Street, CA, 92054"
21632,Weinberger,Alan,Walter,NaN,"P.O. Box 676, South Pasadena, CA, 91031",Physician,G 33359,"00G333590, 1124015011",05/18/2023,Indefinitely effective,"P.O. Box 676, South Pasadena, CA, 91031",South Pasadena,91031,"P.O. Box 676, CA, 91031"
21633,Winters,Julie,Mae,NaN,"6024 Fyler Avenue, Saint Louis, MO, 63139",Registered Nurse,774207,NaN,06/16/2023,Indefinitely effective,"6024 Fyler Avenue, Saint Louis, MO, 63139",Saint Louis,63139,"6024 Fyler Avenue, MO, 63139"
21634,Dondiego,Madelaine,NaN,NaN,"Delano, CA",In-Home Supportive Services Provider,1336006,NaN,07/11/2023,Indefinitely effective,"Delano, CA",Delano,93215,"Delano, CA"


In [78]:
df.to_csv('banned_providers_final.csv', index=False)

In [80]:
zip_latlong = pd.read_csv('zip_to_lat_lon_North America.csv', dtype={'postal code': str})
zip_latlong['postal code'] = zip_latlong['postal code'].astype(str).str.zfill(5)

zip_latlong = zip_latlong[zip_latlong['country code'] == 'US']
zip_latlong = zip_latlong[['postal code','state','province_or_county','latitude','longitude']].reset_index(drop = True)
# #zip_latlong.columns = [str(col) for col in zip_latlong.columns]

zip_latlong

C:\Users\Hernando\AppData\Local\Temp\ipykernel_17060\3375427282.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  zip_latlong = pd.read_csv('zip_to_lat_lon_North America.csv', dtype={'postal code': str})


,postal code,state,province_or_county,latitude,longitude
0,99553,Alaska,Aleutians East,54.1430,-165.7854
1,99571,Alaska,Aleutians East,55.1858,-162.7211
2,99583,Alaska,Aleutians East,54.8542,-163.4113
3,99612,Alaska,Aleutians East,55.0628,-162.3056
4,99661,Alaska,Aleutians East,55.3192,-160.4914
...,...,...,...,...,...
41465,96558,NaN,NaN,19.7542,-155.5858
41466,96598,NaN,NaN,-89.9976,139.2729
41467,96599,NaN,NaN,-77.8460,166.6760
41468,96860,NaN,NaN,21.3448,-157.9774


In [81]:
df['postal code'] = df['Zip Code']

# Convert to string and standardize format (if necessary)
df['postal code'] = df['postal code'].astype(str)
zip_latlong['postal code'] = zip_latlong['postal code'].astype(str)

# Now perform the merge
merged_df = df.merge(zip_latlong, left_on='postal code', right_on='postal code', how='inner')
merged_df

,Last Name,First Name,Middle Name,A/K/A-Also Known As\nD/B/A-Doing Business as,Address(es),Provider Type,License Number,Provider Number,Date of Suspension,Active Period,1st Address,City,Zip Code,Cleaned Address,postal code,state,province_or_county,latitude,longitude
0,111 Pharmacy,NaN,NaN,NaN,"111 West Beverly Blvd., Ste. B, Montebello, CA...",Pharmacy/Pharmacist,41023,"PHA410230, PHA393440, 1821137118",7/8/2016,Indefinitely effective,"111 West Beverly Blvd., Ste. B, Montebello, CA...",Montebello,90640,"111 West Beverly Blvd., Ste. B, Montebello, CA...",90640,California,Los Angeles,34.0133,-118.1130
1,Bonilla,Blanca,NaN,NaN,"828 South Taylor Ave., Unit #5, Montebello, CA",Certified Nurse Assistant,337480,NaN,12/20/2005,Indefinitely effective,"828 South Taylor Ave., Unit #5, Montebello, CA",Montebello,90640,"828 South Taylor Ave., Montebello, CA",90640,California,Los Angeles,34.0133,-118.1130
2,Cal Med Health Plan Inc,NaN,NaN,NaN,"2457 West Beverly Blvd., Montebello, CA; 805 V...",Corporation Owner,NaN,NaN,2/11/2009,Indefinitely effective,"2457 West Beverly Blvd., Montebello, CA",Montebello,90640,"2457 West Beverly Blvd., Montebello, CA",90640,California,Los Angeles,34.0133,-118.1130
3,Chavez,Linda,Martinez,NaN,"1791 Neil Armstrong St., Unit #207, Montebello...",Employee,NaN,NaN,5/20/2003,Indefinitely effective,"1791 Neil Armstrong St., Unit #207, Montebello...",Montebello,90640,"1791 Neil Armstrong St., Montebello, CA",90640,California,Los Angeles,34.0133,-118.1130
4,Gomez,Fabian,NaN,NaN,"711 West Whittier Blvd., Montebello, CA",Physician,NaN,"00C358860, 00C358861",6/6/1994,Indefinitely effective,"711 West Whittier Blvd., Montebello, CA",Montebello,90640,"711 West Whittier Blvd., Montebello, CA",90640,California,Los Angeles,34.0133,-118.1130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16844,Ilangezyan,Lucine,NaN,"Ilangezyan, Lulu; Ilangozyan, Lusine","16326 Ballinger Street, North Hills, CA, 91343",Employee,NaN,NaN,06/20/2023,Indefinitely effective,16326,Unknown,16326,16326,16326,Pennsylvania,Clarion,41.3717,-79.4134
16845,Jizmejian,Gary,Raffi,"Jizmejian, Garry Raffi","27658 Ron Ridge Drive, Santa Clarita, CA, 91350",Employee,NaN,NaN,06/20/2023,Indefinitely effective,27658,Unknown,27658,27658,27658,North Carolina,Wake,35.7977,-78.6253
16846,Lipton,Lisa,Renee,"Weisz, Lisa Renee;\nWeisz Lipton, Lisa Renee","9231 Linkmeadow Lane, Houston, TX, 77025",Other,94026573,NaN,06/23/2023,Indefinitely effective,"9231 Linkmeadow Lane, Houston, TX, 77025",Houston,77025,"9231 Linkmeadow Lane, TX, 77025",77025,Texas,Harris,29.6889,-95.4341
16847,Weinberger,Alan,Walter,NaN,"P.O. Box 676, South Pasadena, CA, 91031",Physician,G 33359,"00G333590, 1124015011",05/18/2023,Indefinitely effective,"P.O. Box 676, South Pasadena, CA, 91031",South Pasadena,91031,"P.O. Box 676, CA, 91031",91031,California,Los Angeles,34.1161,-118.1503


In [95]:
merged_df.to_csv('banned_providers_merged_lat.csv', index=False)
df.to_csv('banned_providers_final.csv', index=False)


In [91]:
#filtering out row that has year of suspension as 1753
df['Date of Suspension'] = pd.to_datetime(df['Date of Suspension'])
df['Year'] = df['Date of Suspension'].dt.year
df = df[df['Year'] != 1753]

merged_df['Date of Suspension'] = pd.to_datetime(merged_df['Date of Suspension'])
merged_df['Year'] = merged_df['Date of Suspension'].dt.year
merged_df = merged_df[merged_df['Year'] != 1753]

In [93]:
merged_df

,Last Name,First Name,Middle Name,A/K/A-Also Known As\nD/B/A-Doing Business as,Address(es),Provider Type,License Number,Provider Number,Date of Suspension,Active Period,1st Address,City,Zip Code,Cleaned Address,postal code,state,province_or_county,latitude,longitude,Year
0,111 Pharmacy,NaN,NaN,NaN,"111 West Beverly Blvd., Ste. B, Montebello, CA...",Pharmacy/Pharmacist,41023,"PHA410230, PHA393440, 1821137118",2016-07-08,Indefinitely effective,"111 West Beverly Blvd., Ste. B, Montebello, CA...",Montebello,90640,"111 West Beverly Blvd., Ste. B, Montebello, CA...",90640,California,Los Angeles,34.0133,-118.1130,2016.0
1,Bonilla,Blanca,NaN,NaN,"828 South Taylor Ave., Unit #5, Montebello, CA",Certified Nurse Assistant,337480,NaN,2005-12-20,Indefinitely effective,"828 South Taylor Ave., Unit #5, Montebello, CA",Montebello,90640,"828 South Taylor Ave., Montebello, CA",90640,California,Los Angeles,34.0133,-118.1130,2005.0
2,Cal Med Health Plan Inc,NaN,NaN,NaN,"2457 West Beverly Blvd., Montebello, CA; 805 V...",Corporation Owner,NaN,NaN,2009-02-11,Indefinitely effective,"2457 West Beverly Blvd., Montebello, CA",Montebello,90640,"2457 West Beverly Blvd., Montebello, CA",90640,California,Los Angeles,34.0133,-118.1130,2009.0
3,Chavez,Linda,Martinez,NaN,"1791 Neil Armstrong St., Unit #207, Montebello...",Employee,NaN,NaN,2003-05-20,Indefinitely effective,"1791 Neil Armstrong St., Unit #207, Montebello...",Montebello,90640,"1791 Neil Armstrong St., Montebello, CA",90640,California,Los Angeles,34.0133,-118.1130,2003.0
4,Gomez,Fabian,NaN,NaN,"711 West Whittier Blvd., Montebello, CA",Physician,NaN,"00C358860, 00C358861",1994-06-06,Indefinitely effective,"711 West Whittier Blvd., Montebello, CA",Montebello,90640,"711 West Whittier Blvd., Montebello, CA",90640,California,Los Angeles,34.0133,-118.1130,1994.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16844,Ilangezyan,Lucine,NaN,"Ilangezyan, Lulu; Ilangozyan, Lusine","16326 Ballinger Street, North Hills, CA, 91343",Employee,NaN,NaN,2023-06-20,Indefinitely effective,16326,Unknown,16326,16326,16326,Pennsylvania,Clarion,41.3717,-79.4134,2023.0
16845,Jizmejian,Gary,Raffi,"Jizmejian, Garry Raffi","27658 Ron Ridge Drive, Santa Clarita, CA, 91350",Employee,NaN,NaN,2023-06-20,Indefinitely effective,27658,Unknown,27658,27658,27658,North Carolina,Wake,35.7977,-78.6253,2023.0
16846,Lipton,Lisa,Renee,"Weisz, Lisa Renee;\nWeisz Lipton, Lisa Renee","9231 Linkmeadow Lane, Houston, TX, 77025",Other,94026573,NaN,2023-06-23,Indefinitely effective,"9231 Linkmeadow Lane, Houston, TX, 77025",Houston,77025,"9231 Linkmeadow Lane, TX, 77025",77025,Texas,Harris,29.6889,-95.4341,2023.0
16847,Weinberger,Alan,Walter,NaN,"P.O. Box 676, South Pasadena, CA, 91031",Physician,G 33359,"00G333590, 1124015011",2023-05-18,Indefinitely effective,"P.O. Box 676, South Pasadena, CA, 91031",South Pasadena,91031,"P.O. Box 676, CA, 91031",91031,California,Los Angeles,34.1161,-118.1503,2023.0


In [ ]:
merged_df.to_csv('banned_providers_merged_lat.csv', index=False)
df.to_csv('banned_providers_final.csv', index=False)